In [1]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.config.load('dj_local_conf.json')
dj.conn()

from workflow.pipeline import model, train, reference, subject, session
from workflow.pipeline.dlc import insert_new_dlc_model, ingest_behavior_videos
from pathlib import Path
import datetime

In [ ]:
# Manual step to insert new models into the model.Model() table
# Set project path to specify the location of the config.yaml file
# When adding new project data, store in dlc_projects folder with new project folder name
# This is the only required paramter
example_project_path = "/home/anaconda/inbox/dlc_projects/Testing2D_Kim-2022-01-04"

# None of the following need to be specified, all are optional parameters
model_prefix = ""
model_description = "Test Model Ingestion for DLC"
paramset_idx = 0

insert_new_dlc_model(project_path=example_project_path, 
                    model_prefix=model_prefix, 
                    paramset_idx=paramset_idx, 
                    model_description=model_description)
        
# Display model.Model() table to see insert
model.Model()

In [ ]:
# Manual step to insert Camera(s) into reference.Device() table
# This step is required for the model.VideoRecording table to be populated
device_id = 0
device_name = 'camera1' 
device_description = 'dlc test camera'
reference_dict = dict(
    device_id=device_id,
    device_name=device_name, 
    device_description=device_description
    )
reference.Device.insert1(reference_dict)
reference.Device()

In [ ]:
# Manual step to insert behavior videos into model.VideoRecording() table
# and into its respective part table model.VideoRecording.File()
# Specify subject and session_id of the behavior videos

# Make sure specified subject exists within the subject.Subject() table  
# Skip this step if subject has been inserted into subject.Subject already
subj_key = dict(
    subject='test_sub',
    subject_nickname="",
    sex='F',
    subject_birth_date=datetime.date(2022, 1, 1),
    subject_description= "Test subject for DLC"
)
subject.Subject.insert1(subj_key)

# Make sure specified session exists in session.Session() and session.SessionDirectory() tables
# Skip these steps if session has been inserted into session tables already
session_key = dict(subject='test_sub', session_id=1,
                   session_datetime='2023-01-07 12:00:00')

session.Session.insert1(session_key)
sdir_key = dict(subject=session_key['subject'], 
                session_id=session_key['session_id'], 
                session_dir='test_sub/session1')
session.SessionDirectory.insert1(sdir_key)

behavior_key = dict(
    subject='test_sub',
    session_id=1
    )   
# Specfiy the device that was used to record the behavior videos
device_id = 0

# Ingest behavior videos for specified behavior key and device_id 
# recording_id is assumed to be 0 (1 recording per session)
ingest_behavior_videos(
    behavior_key, 
    device_id,
    recording_id=0
)

In [ ]:
# Populate automated model.RecordingInfo table containing file metadata
# This step can be run after model.VideoRecording() and model.VideoRecording.File() tables are populated
model.RecordingInfo.populate()

# Display recording info
model.RecordingInfo()

In [ ]:
# Manual Step to insert entry into Pose Estimation Task

# Choose a video recording by manually specifying subject, sesison and recording 
# Or another method
vr_key = dict(subject='test_sub',
            session_id=1,
            recording_id=0)
pe_key_1 = (model.VideoRecording & vr_key).fetch1('KEY')
# Specify Model to be used for Pose Estimation Inference
model_key = (model.Model & 'model_name LIKE "%Testing2D_Kim%"').fetch1('KEY')

pe_key_1.update(model_key)
# pe_key.update(model_key)

task_mode = "trigger" # Can be set to trigger (to trigger inference) 
                      # or load (to load external results into database)

#  Params Optional(default None). Parameters passed to DLC's analyze_videos:
#                 videotype, gputouse, save_as_csv, batchsize, cropping, TFGPUinference,
#                 dynamic, robust_nframes, allow_growth, use_shelve
params={'save_as_csv':True}

# Use built in infer_output_dir function to generate output directory location
# Or can manually specify this  
output_dir = model.PoseEstimationTask.infer_output_dir(pe_key_1)
skip_duplicates = False # Manually set whether duplicate entries should be skipped or retried
model_name = model.Model.fetch1('model_name')
model.PoseEstimationTask.insert_estimation_task(
        video_recording_key=pe_key_1,
        task_mode=task_mode,
        analyze_videos_params=params,
    )
# Display PoseEstimationTask table
model.PoseEstimationTask()

In [ ]:
# Run Model Inference Step to populate model.PoseEstimation table
model.PoseEstimation.populate()